<a href="https://colab.research.google.com/github/Srinath-2603/Srinath/blob/main/pspyk12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cv#user_data

user_id,series,season,timestamp,genre,duration_mins
521,”Mirzapur”,3,2024-07-30 15:00:00,action,300
672,”Panchayat”,3,2024-07-30 15:00:00,comedy,200
197,”Family Man”,2,2024-07-30 15:00:00,action,500
521,”Mirzapur”,2,2024-07-29 15:00:00,action,280
211,”Queens Gambit”,1,2024-07-30 15:00:00,drama,170
521,”Mirzapur”,1,2024-07-28 15:00:00,action,230
844,”Westworld”,3,2024-07-30 15:00:00,sci-fi,310
672,”Panchayat”,3,2024-07-29 15:00:00,comedy,210
256,”Homecoming”,2,2024-07-30 15:00:00,thriller,310
489,”Outer Range”,1,2024-07-30 15:00:00,sci-fi,340
200,”Black Mirror”,2,2024-07-30 15:00:00,sci-fi,140
256,”Outer Range”,2,2024-07-30 15:00:00,thriller,250
489,”Outer Range”,2,2024-07-28 15:00:00,sci-fi,170
200,”Black Mirror”,3,2024-07-29 15:00:00,sci-fi,190
672,”Panchayat”,2,2024-07-28 15:00:00,comedy,160
672,”Outer Range”,1,2024-07-25 15:00:00,sci-fi,250
200,”Black Mirror”,4,2024-07-28 15:00:00,sci-fi,200
844,”Westworld”,2,2024-07-29 15:00:00,sci-fi,300
672,”Black Mirror”,5,2024-07-28 15:00:00,sci-fi,150
672,”Panchayat”,1,2024-07-27 15:00:00,comedy,190

In [1]:
#Import Libraries
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=b3d2949aa20aeb8832393b1fdd0b98f0b58b709e0175fb56649bd348fc60c9ac
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
#Create SparkSession for app (Streaming Analysis)
spark = SparkSession.builder.appName('practice').getOrCreate()

In [ ]:
#Create Dataframe series_df
df = spark.createDataFrame(data=data,schema=schema)

In [ ]:
#Find the user with maximum watchtime
df.groupBy('user_id').agg({'duration_mins':'sum'}).orderBy('sum(duration_mins)',ascending=False).limit(1).show()

In [ ]:
#Calculate overall total Watchtime
df.agg('duration_mins':'sum').show()

In [ ]:
#Find most popular shows (based on watchtime)
df = df.groupBy('series').agg({'duration_mins':'sum'}).orderBy('sum(duration_mins)',ascending = False).show()


In [ ]:
#Find most popular shows (based on user popularity)
df = df.groupBy('series').agg({'user_id':'count'}).orderBy('count(user_id)',ascending = False).show()

In [ ]:
#Find the most popular genre
df = df.groupBy('genre').agg({'duration_mins':'sum'}).orderBy('sum(duration_mins)',ascending = False).show()

In [ ]:
#Calculate total watchtime per use
df.groupBy('user_id').agg({'duration_mins':'sum'}).show()

In [ ]:
#Find most popular genre (based on engagement count)
df = df.groupBy('genre').agg({'user_id':'count'}).orderBy('count(user_id)',ascending = False).show()

In [ ]:
#Find average watchtime per genre
df.groupBy('genre').agg({'duration_min':'avg'})

In [ ]:
#Find peak traffic days
#(Output 1 = Full Date)
df = df.groupBy('timestamp').agg({'timestamp':'count'}).orderBy('count(timestamp)',ascending = False).show()

#(Output 2 = Only Day)



In [ ]:
#Find the user with most diverse show preference
df = df.groupBy('user_id').agg(countDistinct('series')).orderBy(countDistinct('series'),ascending = False).limit(1).show()

In [ ]:
#Find the binge-watchers
df1=df.groupBy('user_id','timestamp','series').agg({'season':'count'}).withColumnRenamed('count(season)','count').orderBy('user_id')
df2=df.groupBy('series').agg({'season':'max'}).withColumnRenamed('max(season)','max')
df3=df1.join(df2,'series','inner')
df3.show()
df3.filter(f.col('count')==f.col('max')).collect()[0][1]

In [ ]:
#Find the user with longest watching streak
df.groupBy('user_id').agg({'duration_mins':'sum'}).orderBy('sum(duration_mins)',ascending = False).limit(1).show()

In [ ]:
#Total Seasons available
df.groupBy('series').agg({'seasons':'max'}).show()

In [ ]:
#Fetch a list of all series
df.rdd.flatMap(lambda x:[x[1]]).distinct().collect()